# 性能调优

在代码可以实现功能且健壮不出错的前提下,我们往往会有优化性能的需求

性能调优大约可以在运行时间和运行内存占用两方面来考量,下面介绍的工具定位精度由粗到细,也分为这两个方面

## 测试整体运行时间

Python中的timeit是测试代码执行效率的工具.可以用命令行直接测试脚本,也可以测试代码字符串的效率,当然最简单的还是直接用ipython的内置timeit魔法命令测某段代码的效率

In [1]:
import timeit
t = timeit.Timer('map(lambda x: x**2,range(1000))')
t.timeit()

0.32536690100096166

In [2]:
!python -m timeit -s "map(lambda x: x**2,range(1000))"

20000000 loops, best of 5: 10.3 nsec per loop


## 函数级性能瓶颈定位

python的标准库中有一个可以实现性能瓶颈定位的模块叫cprofile,他是一个开销极小的C扩展.用它可以实现函数级的性能分析,配合`pstats`模块还可以输出分析报告


### 使用单独模块分析

In [3]:
%%writefile examples/profile/profile_test.py
def foo():
    sum = 0
    for i in range(10000):
        sum += i
    return sum
if __name__=="__main__":
    try :
        import profile
    except:
        import cProfile as profile
        
    profile.run("foo()")

Overwriting examples/profile/profile_test.py


In [4]:
!python examples/profile/profile_test.py

         5 function calls in 0.016 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.001    0.001 :0(exec)
        1    0.016    0.016    0.016    0.016 :0(setprofile)
        1    0.000    0.000    0.001    0.001 <string>:1(<module>)
        1    0.000    0.000    0.016    0.016 profile:0(foo())
        0    0.000             0.000          profile:0(profiler)
        1    0.001    0.001    0.001    0.001 profile_test.py:1(foo)




### 使用命令行分析

In [5]:
%%writefile examples/profile/profile_test_foo.py
#coding:utf-8
def foo():
    sum = 0
    for i in range(10000):
        sum += i
    return sum
if __name__=="__main__":
    foo()

Overwriting examples/profile/profile_test_foo.py


In [6]:
!python -m cProfile examples/profile/profile_test_foo.py

         4 function calls in 0.001 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.001    0.001 profile_test_foo.py:1(<module>)
        1    0.001    0.001    0.001    0.001 profile_test_foo.py:2(foo)
        1    0.000    0.000    0.001    0.001 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




### 统计项说明

统计项|说明
---|---
ncalls| 函数被调用次数
tottime|函数总计运行时间,不含调用函数运行时间
cumtime|函数总计运行时间,含调用的函数运行时间
percall|函数运行一次平均时间,等于tottime(cumtime)/ncalls
`filename:lineno`|函数所在文件名,函数的行号,函数名

### 与pstats结合提供多种形式的报表

In [7]:
%%writefile examples/profile/profile_test_pstats.py
def foo():
    sum = 0
    for i in range(10000):
        sum += i
    return sum
if __name__=="__main__":
    try :
        import profile
    except:
        import cProfile as profile
        
    profile.run("foo()","foo.txt")
    import pstats
    p = pstats.Stats("foo.txt")
    p.sort_stats("time").print_stats()

Overwriting examples/profile/profile_test_pstats.py


In [8]:
!python examples/profile/profile_test_pstats.py

Fri May 26 11:14:16 2023    foo.txt

         5 function calls in 0.001 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.001    0.001 /Users/mac/WORKSPACE/GITHUB/BLOG/TutorialForPython/src/工具链篇/examples/profile/profile_test_pstats.py:1(foo)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
        1    0.000    0.000    0.001    0.001 profile:0(foo())
        1    0.000    0.000    0.001    0.001 :0(exec)
        1    0.000    0.000    0.001    0.001 <string>:1(<module>)
        0    0.000             0.000          profile:0(profiler)




stats有许多函数,可以提供不同的报表

+ stats函数说明

函数|说明
---|---
strip_dirs()|除去文件名前名的路径信息
add(filename,[...])|把profile输出的文件加入stats实例中统计
dump_stats(filename)|把stats统计结果保存到文件
sort_stats(key,[...])|最重要的,可以给profile统计结果排序
reverse_order()|数据反排序
print_stats([restriction,...])|把报表输出到stdout
print_callers([restriction,...])|输出调用指定函数的相关信息
print_callees([restriction,...])|输出指定函数调用过的函数的相关信息

+ sort_stats可接受的参数

参数|说明
---|---
ncalls|被调次数
cumulative|函数运行总时间
file|文件名
module|模块名
pcalls|简单统计
line|行号
name|函数名
nfl|name,file,line
stdname|标准函数名
time|函数内部运行时间

## 语句级性能瓶颈定位

cprofiler只能追踪到哪个函数是性能瓶颈,而函数中哪条语句是性能瓶颈就追踪不到了,对于语句级性能瓶颈定位,python并没有官方工具,但github上有位大神制作了[line_profiler](https://github.com/rkern/line_profiler),这个工具可以实现这一功能,它也几乎可以说是python的半标准工具之一了.

因为不是标准库中的内容,所以需要pip安装.

使用方法十分简单,在需要分析的函数上面加上装饰器`@profile`即可(注意不用import任何东西,这条装饰器在定位好后应该删除以保证代码可以运行)

In [9]:
%%writefile examples/profile/line_profile_test.py

@profile
def foo():
    sum = 0
    for i in range(10000):
        sum += i
    return sum
if __name__=="__main__":
    foo()

Overwriting examples/profile/line_profile_test.py


In [10]:
!python3 -m kernprof -l -v examples/profile/line_profile_test.py

Wrote profile results to line_profile_test.py.lprof
Timer unit: 1e-06 s

Total time: 0.006936 s
File: examples/profile/line_profile_test.py
Function: foo at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profile
     3                                           def foo():
     4         1          0.0      0.0      0.0      sum = 0
     5     10000       3599.0      0.4     51.9      for i in range(10000):
     6     10000       3336.0      0.3     48.1          sum += i
     7         1          1.0      1.0      0.0      return sum



## 内存分析

[memory_profiler](https://pypi.python.org/pypi/memory_profiler/)是用来分析内存使用情况和追踪内存泄露的工具.它用法比较接近`line_profiler`

由于不是标准库中的模块,它需要pip安装.


需要注意的是windows下需要在script文件夹下将`mprof`文件改名为`mprof.py`并在同一目录下创建一个`mprof.bat`文件编辑为如下内容

```shell
@echo off
python "%~dpn0.py" %*
```

它的使用及其简单:

In [11]:
%%writefile examples/profile/memory_test.py
from memory_profiler import profile
@profile
def foo():
    sum = 0
    for i in range(10000):
        sum += i
    return sum
if __name__=="__main__":
    try :
        import profile as cProfile
    except:
        import cProfile 
        
    cProfile.run("foo()","foo.txt")
    import pstats
    p = pstats.Stats("foo.txt")
    p.sort_stats("time").print_stats()

Writing examples/profile/memory_test.py


In [13]:
!python examples/profile/memory_test.py

Filename: /Users/mac/WORKSPACE/GITHUB/BLOG/TutorialForPython/src/工具链篇/examples/profile/memory_test.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     2     42.3 MiB     42.3 MiB           1   @profile
     3                                         def foo():
     4     42.3 MiB      0.0 MiB           1       sum = 0
     5     42.3 MiB      0.0 MiB       10001       for i in range(10000):
     6     42.3 MiB      0.0 MiB       10000           sum += i
     7     42.3 MiB      0.0 MiB           1       return sum


Fri May 26 11:15:00 2023    foo.txt

         115647 function calls (114516 primitive calls) in 1.009 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.585    0.585    0.585    0.585 /Users/mac/WORKSPACE/GITHUB/BLOG/TutorialForPython/src/工具链篇/examples/profile/memory_test.py:2(foo)
     5244    0.054    0.000    0.054    0.000 :0(lstat)
      649    0.047    0.000    0.206    0.000 


就可以看到详细结果了

指定精度可以在profile装饰器后面加上参数 如: @profile(precision=4)

mprof工具类似`kernprof`,用它可以输出更加友好的统计分析页面

In [14]:
%%writefile examples/profile/memory_test_round.py
from memory_profiler import profile
@profile
def foo():
    sum = 0
    for i in range(10000):
        sum += i
    return sum
if __name__=="__main__":
    foo()

Writing examples/profile/memory_test_round.py


In [15]:
!mprof run examples/profile/memory_test_round.py

mprof: Sampling memory every 0.1s
running new process
running as a Python program...
Filename: examples/profile/memory_test_round.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     2     42.5 MiB     42.5 MiB           1   @profile
     3                                         def foo():
     4     42.5 MiB      0.0 MiB           1       sum = 0
     5     42.5 MiB      0.0 MiB       10001       for i in range(10000):
     6     42.5 MiB      0.0 MiB       10000           sum += i
     7     42.5 MiB      0.0 MiB           1       return sum




## 对象分析及追踪(windows下不能用)

[Objgraph](http://mg.pov.lt/objgraph/)可以实现对象分析和追踪,它也是用pip安装,不过它依赖xdot(pip 安装)
和[graphviz](http://www.graphviz.org/)(brew安装)

它可以实现的功能有:

+ 统计
+ 定义过滤对象
+ 遍历和显示对象图

In [23]:
%%writefile examples/profile/Obj_test.py
#encoding=utf-8  
import objgraph  
  
if __name__ == '__main__':  
    x = []  
    y = [x, [x], dict(x=x)]  
    objgraph.show_refs([y], filename='sample-graph.png') #把[y]里面所有对象的引用画出来  
    objgraph.show_backrefs([x], filename='sample-backref-graph.png') #把对x对象的引用全部画出来  
    #objgraph.show_most_common_types() #所有常用类型对象的统计，数据量太大，意义不大  
    objgraph.show_growth(limit=4) #打印从程序开始或者上次show_growth到现在增加的对象（按照增加量的大小排序）  

Writing examples/profile/Obj_test.py


In [24]:
!python examples/profile/Obj_test.py

Graph written to /var/folders/j_/p0q1k_mj4cs0dqn0sqnshsrr0000gn/T/objgraph-q4wvid_g.dot (4 nodes)
Image generated as sample-graph.png
Graph written to /var/folders/j_/p0q1k_mj4cs0dqn0sqnshsrr0000gn/T/objgraph-y50sxedg.dot (6 nodes)
Image generated as sample-backref-graph.png
function               2856     +2856
dict                   1566     +1566
tuple                  1352     +1352
wrapper_descriptor     1128     +1128


于是你可以看到图了

![sample-graph](source/examples/profile/sample-graph.png)
![sample-backref-graph](source/examples/profile/sample-backref-graph.png)